# Multiple dispatch

In this notebook we'll explore **multiple dispatch**, which is a key feature of Julia.

Multiple dispatch makes software *generic* and *fast*!

#### Starting with the familiar

To understand multiple dispatch in Julia, let's start with what we've already seen.

We can declare functions in Julia without giving Julia any information about the types of the input arguments that function will receive:

In [1]:
f(x) = x^2

f (generic function with 1 method)

and then Julia will determine on its own which input argument types make sense and which do not:

In [2]:
f(10)

100

In [3]:
f([1, 2, 3])

MethodError: MethodError: no method matching ^(::Vector{Int64}, ::Int64)
Closest candidates are:
  ^(!Matched::Union{AbstractChar, AbstractString}, ::Integer) at /usr/share/julia/base/strings/basic.jl:721
  ^(!Matched::Rational, ::Integer) at /usr/share/julia/base/rational.jl:475
  ^(!Matched::LinearAlgebra.Diagonal, ::Integer) at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/diagonal.jl:196
  ...

#### Specifying the types of our input arguments

However, we also have the *option* to tell Julia explicitly what types our input arguments are allowed to have.

For example, let's write a function `foo` that only takes strings as inputs.

In [4]:
foo(x::String, y::String) = println("My inputs x and y are both strings!")

foo (generic function with 1 method)

We see here that in order to restrict the type of `x` and `y` to `String`s, we just follow the input argument name by a double colon and the keyword `String`.

Now we'll see that `foo` works on `String`s and doesn't work on other input argument types.

In [5]:
foo("hello", "hi!")

My inputs x and y are both strings!


In [6]:
foo(3, 4)

MethodError: MethodError: no method matching foo(::Int64, ::Int64)

To get `foo` to work on integer (`Int`) inputs, let's tack `::Int` onto our input arguments when we declare `foo`.

In [7]:
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")

foo (generic function with 2 methods)

In [8]:
foo(3, 4)

My inputs x and y are both integers!


Now `foo` works on integers! But look, `foo` also still works when `x` and `y` are strings!

In [9]:
foo("hello", "hi!")

My inputs x and y are both strings!


This is starting to get to the heart of multiple dispatch. When we declared

```julia
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")
```
we didn't overwrite or replace
```julia
foo(x::String, y::String)
```
Instead, we just added an additional ***method*** to the ***generic function*** called `foo`.

A ***generic function*** is the abstract concept associated with a particular operation.

For example, the generic function `+` represents the concept of addition.

A ***method*** is a specific implementation of a generic function for *particular argument types*.

For example, `+` has methods that accept floating point numbers, integers, matrices, etc.

We can use the `methods` to see how many methods there are for `foo`.

In [10]:
methods(foo)

# 2 methods for generic function "foo":
[1] foo(x::String, y::String) in Main at /home/snowztail/Repositories/Introduction-to-Julia/day2/09. Multiple dispatch.ipynb:1
[2] foo(x::Int64, y::Int64) in Main at /home/snowztail/Repositories/Introduction-to-Julia/day2/09. Multiple dispatch.ipynb:1

Aside: how many methods do you think there are for addition?

In [11]:
methods(+)

# 208 methods for generic function "+":
[1] +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87
[2] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigInt) in Base.GMP at gmp.jl:529
[3] +(c::Union{Int16, Int32, Int64, Int8}, x::BigInt) in Base.GMP at gmp.jl:535
[4] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:390
[5] +(c::Union{Int16, Int32, Int64, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:398
[6] +(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:406
[7] +(x::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at /usr/share/julia/stdlib/v1.7/Dates/src/periods.jl:372
[8] +(x::P, y::P) where P<:Dates.Period in Dates at /usr/share/julia/stdlib/v1.7/Dates/src/periods.jl:79
[9] +(x::Dates.Period, y::Dates.Period) in Dates at /usr/share/julia/stdlib/v1.7/Dates/src/periods.jl:361
[10] +(y::Dates.Period, x::Dates.CompoundPeriod) in Dates at /usr/share/julia/stdlib/v1.7/Dates/src/periods.jl:363
[11] +(y::Dates.Period, x::Dates.TimeType) in Dates at /usr/share/julia/stdlib/v1.7/Dates/src/arithmetic.jl:85
[12] +(x::Dates.Period, r::AbstractRange{<:Dates.TimeType}) in Dates at /usr/share/julia/stdlib/v1.7/Dates/src/ranges.jl:62
[13] +(y::Union{Dates.CompoundPeriod, Dates.Period}, x::AbstractArray{<:Dates.TimeType}) in Dates at /usr/share/julia/stdlib/v1.7/Dates/src/deprecated.jl:14
[14] +(A::LinearAlgebra.UnitLowerTriangular, B::LinearAlgebra.LowerTriangular) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/triangular.jl:641
[15] +(A::LinearAlgebra.UnitLowerTriangular, B::LinearAlgebra.UnitLowerTriangular) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/triangular.jl:643
[16] +(UL::LinearAlgebra.UnitLowerTriangular, J::LinearAlgebra.UniformScaling) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/uniformscaling.jl:182
[17] +(A::LinearAlgebra.UnitLowerTriangular, B::LinearAlgebra.Bidiagonal) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:86
[18] +(F::LinearAlgebra.Hessenberg, J::LinearAlgebra.UniformScaling) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/hessenberg.jl:640
[19] +(r::AbstractRange{<:Dates.TimeType}, x::Dates.Period) in Dates at /usr/share/julia/stdlib/v1.7/Dates/src/ranges.jl:63
[20] +(x::AbstractArray{<:Dates.TimeType}, y::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at /usr/share/julia/stdlib/v1.7/Dates/src/deprecated.jl:6
[21] +(A::LinearAlgebra.SymTridiagonal, B::LinearAlgebra.SymTridiagonal) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/tridiag.jl:208
[22] +(A::LinearAlgebra.SymTridiagonal, B::LinearAlgebra.Symmetric) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/symmetric.jl:485
[23] +(A::LinearAlgebra.SymTridiagonal{var"#s857", V} where {var"#s857"<:Real, V<:AbstractVector{var"#s857"}}, B::LinearAlgebra.Hermitian) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/symmetric.jl:487
[24] +(x::LinearAlgebra.SymTridiagonal, H::LinearAlgebra.UpperHessenberg) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/hessenberg.jl:107
[25] +(A::LinearAlgebra.SymTridiagonal, B::LinearAlgebra.Diagonal) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:145
[26] +(A::LinearAlgebra.SymTridiagonal, B::LinearAlgebra.Tridiagonal) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:159
[27] +(A::LinearAlgebra.SymTridiagonal, B::LinearAlgebra.Bidiagonal) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:213
[28] +(A::LinearAlgebra.SymTridiagonal{var"#s859", V} where {var"#s859"<:Number, V<:AbstractVector{var"#s859"}}, B::LinearAlgebra.UniformScaling) in LinearAlgebra at /usr/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:231
[29] +(x::Dates.Instant) in Dates at /usr/share/julia/stdlib/v1.7/Dates/src/arithmetic.jl:4
[30] +(A::LinearAlgebra.B

So, we now can call `foo` on integers or strings. When you call `foo` on a particular set of arguments, Julia will infer the types of the inputs and dispatch the appropriate method. *This* is multiple dispatch.

Multiple dispatch makes our code generic and fast. Our code can be generic and flexible because we can write code in terms of abstract operations such as addition and multiplication, rather than in terms of specific implementations. At the same time, our code runs quickly because Julia is able to call efficient methods for the relevant types.

To see which method is being dispatched when we call a generic function, we can use the @which macro:

In [12]:
@which foo(3, 4)

foo(x::Int64, y::Int64) in Main at /home/snowztail/Repositories/Introduction-to-Julia/day2/09. Multiple dispatch.ipynb:1

Let's see what happens when we use `@which` with the addition operator!

In [13]:
@which 3 + 3

+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87

In [14]:
@which 3 + 3.0

+(x::Number, y::Number) in Base at promotion.jl:379

In [15]:
@which 3.0 + 3

+(x::Number, y::Number) in Base at promotion.jl:379

In [16]:
@which 3.0 + 3.0

+(x::Float64, y::Float64) in Base at float.jl:399

Furthermore, we can extend `+` by defining new methods for it.
First we need to import `+` from `Base`

In [17]:
import Base: +

Let's say we want to be able to concatenate strings using `+`. Without extension, this doesn't work.

In [18]:
"hello " + "world"

MethodError: MethodError: no method matching +(::String, ::String)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at /usr/share/julia/base/operators.jl:655

In [19]:
@which "hello " + "world!"

ErrorException: no unique matching method found for the specified argument types

So we add a method for `+` that takes two strings as inputs and concatenates them.

In [20]:
+(x::String, y::String) = string(x,y)

+ (generic function with 209 methods)

In [21]:
?string

ErrorException: syntax: invalid identifier name "?"

In [22]:
"hello " + "world!"

"hello world!"

Success! And furthermore now we can prove to ourselves that Julia has dispatched on the types of "hello " and "world" to choose the appropriate method

In [ ]:
@which "hello " + "world!"

And we can continue to add other methods to our generic function `foo`. Let's add one that takes the ***abstract type*** `Number`, which includes subtypes such as `Int`, `Float64`, and other objects you would think of as numbers:

In [23]:
foo(x::Number, y::Number) = println("My inputs x and y are both numbers!")

foo (generic function with 3 methods)

This method for `foo` will work on, for example, floating point numbers:

In [24]:
foo(3.0, 4.0)

My inputs x and y are both numbers!


In [25]:
foo(3,4)

My inputs x and y are both integers!


We can also add a fallback, duck-typed method for `foo` that takes inputs of any type:

In [26]:
foo(x, y) = println("I accept inputs of any type!")

foo (generic function with 4 methods)

In [27]:
methods(foo)

# 4 methods for generic function "foo":
[1] foo(x::String, y::String) in Main at /home/snowztail/Repositories/Introduction-to-Julia/day2/09. Multiple dispatch.ipynb:1
[2] foo(x::Int64, y::Int64) in Main at /home/snowztail/Repositories/Introduction-to-Julia/day2/09. Multiple dispatch.ipynb:1
[3] foo(x::Number, y::Number) in Main at /home/snowztail/Repositories/Introduction-to-Julia/day2/09. Multiple dispatch.ipynb:1
[4] foo(x, y) in Main at /home/snowztail/Repositories/Introduction-to-Julia/day2/09. Multiple dispatch.ipynb:1

Given the methods we've already written for `foo` so far, this method will be called whenever we pass non-numbers to `foo`:

In [28]:
v = rand(3)
foo(v, v)

I accept inputs of any type!


### Exercises

#### 9.1

Extend the function `foo`, adding a method that takes only one input argument, which is of type `Bool`, and returns "foo with one boolean!"

In [37]:
foo(x::Bool) = println("foo with one boolean!")

foo with one boolean!


foo (generic function with 5 methods)

#### 9.2

Check that the method being dispatched when you execute 
```julia
foo(true)
```
is the one you wrote.

In [38]:
foo(true)

foo with one boolean!


In [43]:
@assert foo(true) == "foo with one boolean!"

AssertionError: AssertionError: foo(true) == "foo with one boolean!"